# Company Problem

In this notebook, our goal is to familiarize you with Logica while also enabling you to contrast Logica with SQL.

### Package Import

You can disregard the message when executing the cell below

In [ ]:
from logica import colab_logica
import pandas as pd
import sqlite3

In [ ]:
%load_ext sql

### Database Creation

Logica is typically run on a database. Since we only possess CSV files, we need first create a database.

In [ ]:
Dep=pd.read_csv("datasets/Department.csv")
Dpd=pd.read_csv("datasets/Dependent.csv")
Dep_loc=pd.read_csv("datasets/Dept_Location.csv")
Emp=pd.read_csv("datasets/Employee.csv")
Proj=pd.read_csv("datasets/Project.csv")
Works_On=pd.read_csv("datasets/Works_On.csv")

In [ ]:
conn = sqlite3.connect('my_database.db')

Dep.to_sql('Department', conn, if_exists='replace', index=False)
Dpd.to_sql('Dependent', conn, if_exists='replace', index=False)
Dep_loc.to_sql('Dept_Location', conn, if_exists='replace', index=False)
Emp.to_sql('Employee', conn, if_exists='replace', index=False)
Proj.to_sql('Project', conn, if_exists='replace', index=False)
Works_On.to_sql('Works_On', conn, if_exists='replace', index=False)

conn.close()

### Queries

We've provided a range of queries, in SQL and Logica formats, sourced from the textbook "FUNDAMENTALS OF Database Systems". Consider these as your starting point. Enjoy!

#### Query 0: Retrieve the birth date and address of the employee(s) whose name is 'John B. Smith'.

$\mathrm{\sigma_{Fname='John' \land Minit='B' \land Lname='Smith'}(EMPLOYEE)}$

In [ ]:
%%sql sqlite:///my_database.db
SELECT Bdate, Address
FROM EMPLOYEE
WHERE Fname='John' AND Minit='B' AND Lname='Smith'

In [ ]:
%%logica Q0_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

Q0_Result(Bdate:x, Address:y) :- 
    Employee(Bdate:x, Address:y, Fname:"John", Minit:"B", Lname:"Smith");

#### Query 1: Retrieve the name and address of all employees who work for the 'Research' department. 

$\mathrm{\Pi_{Fname, Lname, Address}(\sigma_{Dname='Research'}(DEPARTMENT\bowtie_{Dnumber=Dno}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db
SELECT Fname,Lname,Address
FROM EMPLOYEE,DEPARTMENT
WHERE Dname='Research' AND Dnumber=Dno;

In [ ]:
%%logica Q1_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");


Q1_Result(Fname:fn, Lname:ln, address:add):- 
    Employee(Fname:fn, Lname:ln, Address:add, Dno:x), 
    Department(Dname:"Research", Dnumber:y),
    x=y;

#### Query 2: For every project located in 'Stafford', list the project number, the controlling department number, and the department manager's last name, address, and birth date. 

$\mathrm{\Pi_{Pnumber,Dnum,Lname,Address,Bdate}(((\sigma_{Plocation='Stafford'}(PROJECT))
\bowtie_{Dnum=Dnumber}(DEPARTMENT))\bowtie_{Mgr\_ssn=Ssn}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db
Select Pnumber,Dnum,Lname,Address,Bdate
From Project,Department,Employee
Where Dnum=Dnumber And Mgr_ssn=Ssn And Plocation='Stafford'

In [ ]:
%%logica Q2_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");


Q2_Result(Pnumber:pn, Dnum:x, Lname:ln, Address:add, Bdate:bd):-
    Project(Pnumber:pn, Dnum:x, Plocation: "Stafford"), 
    Department(Dnumber:y, Mgr_ssn:ssn_1),
    Employee(Lname:ln, Address:add, Ssn:ssn_2, Bdate:bd),
    x=y, ssn_1=ssn_2;

#### Query 3: Who is the boss? The manager of the department that an employee works for!

$\mathrm{\Pi_{SSN, Lname, Address}(EMPLOYEE\bowtie_{Super\_Ssn=Ssn}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db

SELECT E.SSN as 'Employee', 
       M.SSN as 'Boss'
FROM   Employee E, 
       Employee M
WHERE E.Super_SSN = M.SSN
ORDER BY M.SSN;

In [ ]:
%%logica Q3_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

Q3_Result(Employee:y,Boss:x):- 
    Employee(Super_ssn:x),Employee(Ssn:y), x=y;

#### Query 4: Make a list of all project numbers for projects that involve an employee whose last name is ‘Smith’, either as a worker or as a manager of the department that controls the project.

$\mathrm{SMITHS(Essn) \leftarrow \pi_{Ssn}(\sigma_{Lname='Smith'}(EMPLOYEE))}$

$\mathrm{SMITH\_WORKER\_PROJS \leftarrow \pi_{Pno}(WORKS\_ON) \ast SMITHS}$

$\mathrm{MGRS \leftarrow \pi_{Lname,Dnumber}(EMPLOYEE \bowtie_{Ssn=Mgr\_ssn}DEPARTMENT)}$

$\mathrm{SMITH\_MANAGED\_DEPTS(Dnum) \leftarrow \pi_{Dnumber}(\sigma_{Lname='Smith'}(MGRS))}$

$\mathrm{SMITH\_MANAGED\_PROJS(Pno) \leftarrow \pi_{Pnumber}(SMITH\_MANAGED\_DEPTS * PROJECT)}$

$\mathrm{RESULT \leftarrow (SMITH\_WORKER\_PROJS \cup SMITH\_MANAGED\_PROJS)}$

In [ ]:
%%sql sqlite:///my_database.db

SELECT DISTINCT P.Pnumber
FROM Project AS P
JOIN Department AS D ON P.Dnum = D.Dnumber
JOIN Employee AS E ON D.Mgr_ssn = E.ssn
WHERE E.Lname = 'Smith'

UNION

SELECT DISTINCT P.Pnumber
FROM Project AS P
JOIN Works_on AS W ON P.Pnumber = W.Pno
JOIN Employee AS E ON W.Essn = E.Ssn
WHERE E.Lname = 'Smith'

In [ ]:
%%logica Q4_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

Q4_Result(Pnumber:pno):-
    Works_On(Essn:x, Pno:pno),
    Employee(Lname:"Smith",Ssn:y),
    x=y;
    
Q4_Result(Pnumber:pno):-
    Project(Pnumber:pno, Dnum:x),
    Department(Dnumber:y,Mgr_ssn:ssn_1),
    Employee(Ssn:ssn_2, Lname:"Smith"),
    x=y,
    ssn_1=ssn_2;

#### Query 5: For each employee, retrieve the employee’s first and last name and the first and last name of his or her immediate supervisor.

$\mathrm{\Pi_{E\_fname, E\_lname, S\_fname, S\_lname}(EMPLOYEE\bowtie_{Super\_Ssn=Ssn}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db

SELECT E.Fname as E_fname, E.Lname as E_lname, S.Fname as S_fname, S.Lname as S_lname
FROM EMPLOYEE AS E, EMPLOYEE AS S
WHERE E.Super_ssn = S.Ssn;

In [ ]:
%%logica Q5_Result

@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

Q5_Result(E_fname:fn_x, E_lname:ln_x, S_fname:fn_y,S_lname:ln_y):- 
    Employee(Fname:fn_x, Lname:ln_x,Super_ssn:x),
    Employee(Fname:fn_y, Lname:ln_y,Ssn:y), 
    x=y;

#### Query 6: Select all combinations of EMPLOYEE Ssn and DEPARTMENT Dname (Q10) in the database

$\mathrm{\sigma_{Ssn, Dname}EMPLOYEE \times DEPARTMENT}$

In [ ]:
%%sql sqlite:///my_database.db
    
SELECT Ssn, Dname
FROM EMPLOYEE, DEPARTMENT

In [ ]:
%%logica Q6_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

Q6_Result(Ssn:x, Dname:y):-
    Employee(Ssn:x), Department(Dname:y);

#### Query 7: Show the resulting salaries if every employee working on the ‘ProductX’ project is given a 10% raise

$\mathrm{EMP\_PROJX \leftarrow \sigma_{Pname='ProjectX'}(WORKS\_ON \bowtie_{Pno=Pnumber} PROJECT)}$

$\mathrm{RESULT(Fname, Lname, Increased\_sal) \leftarrow \pi_{Fname, Lname, 1.1*Salary}(EMPLOYEE \bowtie_{Ssn=Essn} EMP\_PROJX)}$

In [ ]:
%%sql sqlite:///my_database.db
    
SELECT E.Fname, E.Lname, 1.1 * E.Salary AS Increased_sal
FROM EMPLOYEE AS E, WORKS_ON AS W, PROJECT AS P
WHERE E.Ssn = W.Essn AND W.Pno = P.Pnumber AND
P.Pname = 'ProductX';

In [ ]:
%%logica Q7_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

Q7_Result(Fname:fn, Lname:ln, Increased_Sal:sl*1.1):-
    Employee(Fname:fn, Lname:ln,SSn:ssn1, Salary:sl),
    Works_On(Essn:ssn2, Pno:x), 
    Project(Pname:"ProductX",Pnumber:y),
    ssn1=ssn2, x=y;

## Logica Exercises

#### Query 8: Retrieve the name and address of all employees who work for the 'Administration' department. 

In [ ]:
%%logica Q8_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

# Insert your logica query here

#### Query 9: Retrieve the name of each employee who has a dependent with the same sex as the employee.

In [ ]:
%%logica Q9_Result
@Engine("sqlite");
@AttachDatabase("mydata", "my_database.db");

# Insert your logica query here